# Summary
This notebook apply the proposed method (GMM clustering) to a set of microarray datasets.

In [1]:
import sys
sys.path.append("..")

#GPU configuration
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default


import random
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scripts.data_generator as data_generator
import scripts.feature_ranking as feature_ranking
import scripts.features_2d as features_2d
import scripts.ga as ga
import scripts.preprocess as preprocess
import scripts.ga_evaluation as ga_evaluation
import scripts.bio_analysis as bio_analysis
import tensorflow as tf
from IPython import get_ipython
from tqdm import tqdm
from collections import Counter
import seaborn as sns
import time


plt.ion()
plt.show()

random_state=1
random.seed( random_state )
np.random.seed(random_state)


%load_ext autoreload
%autoreload 2

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


# Steps
- Clustering 1d to select best discriminant features

- Clustering 2d to select redundant, close and outlier features

In [2]:
random_state=0
random.seed( random_state )
np.random.seed(random_state)

In [6]:
results = None
filenames = np.array(['alon', 'borovecki', 'chiaretti', 'christensen', 'golub', 'gordon',
       'khan', 'sorlie', 'su', 'yeoh', 'west'])
clustering = "gmm"
path = '../data/microarray/'
method = "adapted_ratkowsky_lance"
imp_f = np.arange(20)
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values


    n_clusters = len(np.unique(truth))

    z_file= f"../data/microarray/Z_{name}_correlation.npy"
    print(f"\n##########  {name}, {data.shape}")

    # Clustering 1D
    meta_features = feature_ranking.rank_features(data,
                                              nb_bins=20,
                                              rank_threshold=90,
                                              z_file=z_file,
                                              metric='correlation',
                                              redundant_threshold=0.4)
    t2 = time.time()
    print(f"TIME: 1d Features : {(t2-t1)/60} min")

#     model_file = f'../models/gmm_arl.h5' 

#     population, n = features_2d.run(data,
#                                 n_clusters,
#                                 meta_features,
#                                 model_file=model_file,
#                                 add_close_population=False)
    t3 = time.time()
    t4 = time.time()
    print(f"TIME: 2d scores: {(t4-t3)/60} min")
    round_size = 3
    epochs = 10*round_size

    sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
    }
#     sampling = {
#         "ARCHIVE2D": { 
#             "ga": 0.25,
#             "max": 0.25 },
#         "CLOSE": { 
#             "ga": 0.4,
#             "max": 0.4 },
#         "IMP1D": { 
#             "ga": 0.25,
#             "max": 0.25 },
#         "RANDOM": { 
#             "ga": 0.1,
#             "max": 0.1},
#         }
    params = ga.ga_parameters(
        n_clusters,
        data.shape[1],
        truth,
        meta_features,
        method=method,
        truth_methods=['ari'],
        archive_2d=None,#population[:data.shape[1] // 4],
        epochs=epochs,
        sampling = sampling,
        round_size=round_size,
        allow_subspace_overlap = True,
        improvement_per_mutation_report = False,
        clustering = clustering
        
    )
    solutions, archive= ga.run(data, params)
#     display(params["report"].groupby(["op", "improvement"]).count())
    solutions["dataset_name"] = name
    
    t5 = time.time()
    n_total = t5-t1
    print(f"TIME: GA: {(t5-t4)/60} min")
    print(f"TIME: Total: {(t5-t1)/60} min")
    solutions["total_time"] = round((t5-t1)/60, 1)
    solutions["t(feature_sel)"] = round((t2-t1)/60, 1)
    solutions["t(cnn)"] = round((t3-t2)/60, 1)
    solutions["t(clustering2d)"] = round((t4-t3)/60, 1)
    solutions["t(ga)"] = round((t5-t4)/60, 1)
    
    solutions["n_ga"] = archive.shape[0]
    solutions["n_cnn"] = n_total
    solutions["input_size"] = data.shape[1]
    
    
    solutions["n_non_redundant"] = meta_features[(meta_features["redundant"] ==1) ].shape[0]
    solutions["n_imp"] = meta_features[(meta_features["relevance"] !=0)].shape[0]
    solutions["n_imp4"] = meta_features[(meta_features["relevance"] ==4)].shape[0]
    solutions["n_imp3"] = meta_features[(meta_features["relevance"] ==3)].shape[0]
    
    
    if results is None: 
        results = solutions
    else:
        results = pd.concat([results, solutions], ignore_index = True)
    results.to_excel(f"../reports/microarray_{clustering}_{method}.xlsx")
    


##########  singh, (102, 12600)
*** Computing 1D feature ranking ...
Dispersion tests took 0.42 sec
Entropy computation 1.9 sec
KNN computation 9.59 sec
Sorting and thresholds 0.04 sec
Performing hierarchical clustering...
Hierarchical clustering 6.96 sec


  0%|          | 0/31 [00:00<?, ?it/s]

Handle redundant features 1.51 sec
Returning 6021 redundant features and  735 important features
TIME: 1d Features : 0.34079992373784385 min
TIME: 2d scores: 7.947285970052083e-09 min
*** Optimization algorithm 
Non redundant features 6579, orig size 12600, nb imp : 735
adding 50 random population


 10%|▉         | 3/31 [00:09<01:27,  3.14s/it]

,adapted_ratkowsky_lance,ari,features,nmi,partition,point_biserial,silhouette,size,structure
0,0.32,0.04,"[240, 658, 690, 716, 888, 933, 1146, 1175, 126...",0.06,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...",2.18,0.66,165.0,"Counter({0: 82, 1: 20})"


...

 97%|█████████▋| 30/31 [03:25<00:09,  9.94s/it]

,adapted_ratkowsky_lance,ari,features,nmi,partition,point_biserial,silhouette,size,structure
0,0.16,-0.04,"[87, 5992, 9336, 12493]",0.01,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...",0.16,0.43,4.0,"Counter({1: 126, 0: 47})"
1,0.17,0.01,"[831, 4452, 11260]",0.00,"[0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, ...",0.17,0.49,3.0,"Counter({0: 137, 1: 36})"
2,0.19,0.04,"[3872, 6261, 6271]",0.01,"[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, ...",0.14,0.46,3.0,"Counter({0: 114, 1: 59})"
3,0.20,0.03,"[7277, 8864]",0.00,"[1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, ...",0.12,0.53,2.0,"Counter({1: 126, 0: 47})"
4,0.19,-0.04,"[6293, 6908, 8502, 11619]",0.01,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...",0.15,0.44,4.0,"Counter({0: 122, 1: 51})"
5,0.20,0.08,"[5324, 8375]",0.03,"[1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, ...",0.13,0.53,2.0,"Counter({0: 117, 1: 56})"
6,0.18,-0.02,"[409, 512, 1674, 1675, 7609]",0.01,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, ...",0.15,0.38,5.0,"Counter({0: 105, 1: 68})"
7,0.20,-0.00,"[405, 1744, 2242, 2243, 4990]",0.00,"[1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",0.20,0.46,5.0,"Counter({1: 121, 0: 52})"
8,0.19,0.01,"[1027, 1345, 1590, 1727, 1933, 2006, 2162, 397...",0.00,"[1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, ...",0.35,0.34,34.0,"Counter({0: 114, 1: 59})"
9,0.20,0.03,"[851, 3136]",0.02,"[0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, ...",0.12,0.50,2.0,"Counter({0: 95, 1: 78})"


adding 7 random population


100%|██████████| 31/31 [03:36<00:00,  6.97s/it]


TIME: GA: 3.6023046493530275 min
TIME: Total: 4.126767635345459 min


In [21]:
results = pd.read_excel("../reports/microarray_gmm_adapted_ratkowsky_lance.xlsx")

max_ari = results.groupby("dataset_name").agg({"ari": max}).reset_index()

pd.merge(results[["dataset_name", "ari", "silhouette"]], 
         max_ari, on = ["dataset_name", "ari"]).groupby("dataset_name").max()

,ari,silhouette
dataset_name,,
alon,0.15,0.44
borovecki,0.54,0.52
chiaretti,0.16,0.16
christensen,1.00,0.63
golub,0.42,0.52
gordon,0.86,0.60
gravier,0.11,0.57
khan,0.65,0.33
singh,0.05,0.66


In [4]:
results = None
filenames = np.array([
    'alon', 'borovecki', 'chiaretti', 'christensen', 'golub', 'gordon',
       'khan', 'sorlie', 'su', 'yeoh', 'west'])
clustering = "hdbscan"
path = '../data/microarray/'
method = "adapted_ratkowsky_lance"
imp_f = np.arange(20)
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values


    n_clusters = len(np.unique(truth))

    z_file= f"../data/microarray/Z_{name}_correlation.npy"
    print(f"\n##########  {name}, {data.shape}")

    # Clustering 1D
    meta_features = feature_ranking.rank_features(data,
                                              nb_bins=20,
                                              rank_threshold=90,
                                              z_file=z_file,
                                              metric='correlation',
                                              redundant_threshold=0.4)
    t2 = time.time()
    print(f"TIME: 1d Features : {(t2-t1)/60} min")

    t3 = time.time()
    t4 = time.time()
    print(f"TIME: 2d scores: {(t4-t3)/60} min")
    round_size = 3
    epochs = 10*round_size

    sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
    }

    params = ga.ga_parameters(
        n_clusters,
        data.shape[1],
        truth,
        meta_features,
        method=method,
        truth_methods=['ari'],
        archive_2d=None,#population[:data.shape[1] // 4],
        epochs=epochs,
        sampling = sampling,
        round_size=round_size,
        allow_subspace_overlap = True,
        improvement_per_mutation_report = False,
        clustering = clustering
        
    )
    solutions, archive= ga.run(data, params)
#     display(params["report"].groupby(["op", "improvement"]).count())
    solutions["dataset_name"] = name
    
    t5 = time.time()
    n_total = t5-t1
    print(f"TIME: GA: {(t5-t4)/60} min")
    print(f"TIME: Total: {(t5-t1)/60} min")
    solutions["total_time"] = round((t5-t1)/60, 1)
    solutions["t(feature_sel)"] = round((t2-t1)/60, 1)
    solutions["t(cnn)"] = round((t3-t2)/60, 1)
    solutions["t(clustering2d)"] = round((t4-t3)/60, 1)
    solutions["t(ga)"] = round((t5-t4)/60, 1)
    
    solutions["n_ga"] = archive.shape[0]
    solutions["n_cnn"] = n_total
    solutions["input_size"] = data.shape[1]
    
    
    solutions["n_non_redundant"] = meta_features[(meta_features["redundant"] ==1) ].shape[0]
    solutions["n_imp"] = meta_features[(meta_features["relevance"] !=0)].shape[0]
    solutions["n_imp4"] = meta_features[(meta_features["relevance"] ==4)].shape[0]
    solutions["n_imp3"] = meta_features[(meta_features["relevance"] ==3)].shape[0]
    
    
    if results is None: 
        results = solutions
    else:
        results = pd.concat([results, solutions], ignore_index = True)
    results.to_excel(f"../reports/microarray_{clustering}_{method}.xlsx")
results.groupby("dataset_name").agg({"ari": max})


##########  west, (49, 6858)
*** Computing 1D feature ranking ...
Dispersion tests took 0.19 sec
Entropy computation 0.87 sec
KNN computation 1.43 sec
Sorting and thresholds 0.11 sec
Loading clustering from file
Hierarchical clustering 0.02 sec


  0%|          | 0/31 [00:00<?, ?it/s]

Handle redundant features 1.13 sec
Returning 1530 redundant features and  198 important features
TIME: 1d Features : 0.0629282792409261 min
TIME: 2d scores: 3.973642985026042e-09 min
*** Optimization algorithm 
Non redundant features 5328, orig size 6858, nb imp : 198
adding 50 random population


 10%|▉         | 3/31 [00:05<00:50,  1.82s/it]

,adapted_ratkowsky_lance,ari,features,nmi,partition,point_biserial,silhouette,size,structure
0,0.17,0.06,"[4062, 4761]",0.08,"[-1, -1, 1, -1, 1, 1, 0, 0, 1, 1, 1, 0, -1, 1,...",0.01,0.58,2.0,"Counter({1: 19, -1: 16, 0: 14})"


...

,adapted_ratkowsky_lance,ari,features,nmi,partition,point_biserial,silhouette,size,structure
0,0.17,0.06,"[4062, 4761]",0.08,"[-1, -1, 1, -1, 1, 1, 0, 0, 1, 1, 1, 0, -1, 1,...",0.01,0.58,2.0,"Counter({1: 19, -1: 16, 0: 14})"
1,0.12,-0.02,"[24, 383, 450, 947, 1115, 1219, 1419, 1443, 19...",0.01,"[0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0...",-0.19,0.26,27.0,"Counter({0: 23, 1: 14, -1: 12})"
2,0.16,0.01,"[42, 291, 947, 1119, 1248, 1402, 1419, 2041, 2...",0.04,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.07,0.37,21.0,"Counter({0: 30, 1: 13, -1: 6})"
3,0.09,-0.01,"[126, 553]",0.01,"[-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, -1, ...",0.02,0.45,2.0,"Counter({0: 24, -1: 13, 1: 12})"
4,0.21,0.01,"[2694, 3401, 3580, 4694, 5536, 6779]",0.03,"[0, 0, 0, 1, 1, 1, -1, -1, 0, 0, -1, 1, 1, 1, ...",0.11,0.61,6.0,"Counter({0: 31, 1: 13, -1: 5})"
5,0.11,-0.02,"[1269, 5593, 6515]",0.00,"[0, 0, 0, 0, 0, 0, 0, 1, -1, 0, -1, 0, -1, 1, ...",-0.01,0.36,3.0,"Counter({-1: 23, 0: 15, 1: 11})"
6,0.16,0.01,"[516, 2094, 2529]",0.02,"[0, 0, 0, 0, 0, 0, -1, -1, -1, 1, -1, 0, -1, 0...",0.05,0.58,3.0,"Counter({0: 23, 1: 16, -1: 10})"
7,0.15,-0.02,"[3395, 4726, 5179, 5356, 5400, 5401]",0.00,"[-1, -1, 0, 0, 1, 1, -1, -1, 1, 1, 1, 0, 0, 1,...",0.01,0.37,6.0,"Counter({1: 25, 0: 13, -1: 11})"
8,0.12,0.03,"[2772, 3462, 3673]",0.04,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0.03,0.45,3.0,"Counter({0: 27, 1: 12, -1: 10})"
9,0.16,-0.02,"[745, 6514]",0.00,"[0, 0, 0, 1, 1, 1, 0, 0, 1, -1, -1, 1, 1, 0, -...",0.06,0.56,2.0,"Counter({1: 25, 0: 15, -1: 9})"


adding 6 random population


100%|██████████| 31/31 [01:35<00:00,  3.07s/it]


TIME: GA: 1.5861598571141562 min
TIME: Total: 1.6490885535875957 min


,ari
dataset_name,
alon,0.16
borovecki,0.00
chiaretti,0.16
christensen,0.82
golub,0.27
gordon,0.27
khan,0.34
sorlie,0.13
su,0.37


In [5]:
results = pd.read_excel("../reports/microarray_hdbscan_adapted_ratkowsky_lance.xlsx")

max_ari = results.groupby("dataset_name").agg({"ari": max}).reset_index()

pd.merge(results[["dataset_name", "ari", "silhouette"]], 
         max_ari, on = ["dataset_name", "ari"]).groupby("dataset_name").max()

,ari,silhouette
dataset_name,,
alon,0.16,0.31
borovecki,0.00,0.00
chiaretti,0.16,0.25
christensen,0.82,0.73
golub,0.27,0.20
gordon,0.27,0.11
khan,0.34,0.41
sorlie,0.13,0.03
su,0.37,0.37


In [16]:
results = None
filenames = np.array(['alon', 'borovecki', 'chiaretti', 'christensen', 'golub', 'gordon',
       'khan', 'sorlie', 'su', 'yeoh'])
clustering = "leiden"
path = '../data/microarray/'
method = "adapted_ratkowsky_lance"
imp_f = np.arange(20)
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values


    n_clusters = len(np.unique(truth))

    z_file= f"../data/microarray/Z_{name}_correlation.npy"
    print(f"\n##########  {name}, {data.shape}")

    # Clustering 1D
    meta_features = feature_ranking.rank_features(data,
                                              nb_bins=20,
                                              rank_threshold=90,
                                              z_file=z_file,
                                              metric='correlation',
                                              redundant_threshold=0.4)
    t2 = time.time()
    print(f"TIME: 1d Features : {(t2-t1)/60} min")

    t3 = time.time()
    t4 = time.time()
    print(f"TIME: 2d scores: {(t4-t3)/60} min")
    round_size = 3
    epochs = 10*round_size

    sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
    }
#     sampling = {
#         "ARCHIVE2D": { 
#             "ga": 0.25,
#             "max": 0.25 },
#         "CLOSE": { 
#             "ga": 0.4,
#             "max": 0.4 },
#         "IMP1D": { 
#             "ga": 0.25,
#             "max": 0.25 },
#         "RANDOM": { 
#             "ga": 0.1,
#             "max": 0.1},
#         }
    params = ga.ga_parameters(
        n_clusters,
        data.shape[1],
        truth,
        meta_features,
        method=method,
        truth_methods=['ari'],
        archive_2d=None,#population[:data.shape[1] // 4],
        epochs=epochs,
        sampling = sampling,
        round_size=round_size,
        allow_subspace_overlap = True,
        improvement_per_mutation_report = False,
        clustering = clustering
        
    )
    solutions, archive= ga.run(data, params)
#     display(params["report"].groupby(["op", "improvement"]).count())
    solutions["dataset_name"] = name
    
    t5 = time.time()
    n_total = t5-t1
    print(f"TIME: GA: {(t5-t4)/60} min")
    print(f"TIME: Total: {(t5-t1)/60} min")
    solutions["total_time"] = round((t5-t1)/60, 1)
    solutions["t(feature_sel)"] = round((t2-t1)/60, 1)
    solutions["t(cnn)"] = round((t3-t2)/60, 1)
    solutions["t(clustering2d)"] = round((t4-t3)/60, 1)
    solutions["t(ga)"] = round((t5-t4)/60, 1)
    
    solutions["n_ga"] = archive.shape[0]
    solutions["n_cnn"] = n_total
    solutions["input_size"] = data.shape[1]
    
    
    solutions["n_non_redundant"] = meta_features[(meta_features["redundant"] ==1) ].shape[0]
    solutions["n_imp"] = meta_features[(meta_features["relevance"] !=0)].shape[0]
    solutions["n_imp4"] = meta_features[(meta_features["relevance"] ==4)].shape[0]
    solutions["n_imp3"] = meta_features[(meta_features["relevance"] ==3)].shape[0]
    
    
    if results is None: 
        results = solutions
    else:
        results = pd.concat([results, solutions], ignore_index = True)
    results.to_excel("../reports/microarray_leiden.xlsx")
    


##########  alon, (62, 1991)
*** Computing 1D feature ranking ...
Dispersion tests took 0.01 sec
Entropy computation 0.3 sec
KNN computation 0.13 sec
Sorting and thresholds 0.01 sec
Performing hierarchical clustering...


  0%|          | 0/31 [00:00<?, ?it/s]

Hierarchical clustering 0.1 sec
Handle redundant features 0.15 sec
Returning 1520 redundant features and  121 important features
TIME: 1d Features : 0.011996614933013915 min
TIME: 2d scores: 3.973642985026042e-09 min
*** Optimization algorithm 
Non redundant features 471, orig size 1991, nb imp : 121
adding 50 random population


 10%|▉         | 3/31 [00:24<03:02,  6.50s/it]

,adapted_ratkowsky_lance,ari,features,nmi,partition,point_biserial,silhouette,size,structure
0,0.18,-0.01,"[22, 80, 101, 108, 116, 156, 171, 213, 215, 24...",0.0,"[0, 1, 0, 1, 2, 2, 0, 2, 1, 0, 0, 1, 0, 2, 0, ...",0.45,0.21,58.0,"Counter({0: 25, 1: 23, 2: 14})"


...

 97%|█████████▋| 30/31 [29:33<00:56, 56.54s/it]

,adapted_ratkowsky_lance,ari,features,nmi,partition,point_biserial,silhouette,size,structure
0,0.15,-0.00,"[1436, 3733, 4371, 4460, 4675, 5771, 7539, 868...",0.01,"[0, 0, 0, 2, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 2, ...",0.13,0.18,13.0,"Counter({0: 87, 1: 86, 2: 75})"
1,0.17,0.54,"[0, 2, 18, 26, 28, 46, 55, 71, 564, 1009, 1046...",0.60,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.30,0.29,27.0,"Counter({0: 87, 1: 83, 2: 78})"
2,0.13,0.22,"[3, 4, 15, 19, 23, 120, 158, 306, 465, 519, 53...",0.37,"[1, 1, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 2, 2, ...",0.34,0.16,44.0,"Counter({0: 83, 1: 62, 2: 60, 3: 43})"
3,0.12,0.33,"[1, 14, 197, 358, 752, 1366, 1694, 1755, 1871,...",0.48,"[2, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, ...",0.24,0.25,31.0,"Counter({0: 92, 1: 66, 2: 63, 3: 27})"
4,0.12,-0.00,"[1198, 3272, 4463, 4491, 5716, 6420, 7566, 822...",0.01,"[0, 2, 0, 1, 1, 1, 1, 0, 0, 2, 0, 0, 2, 1, 2, ...",0.10,0.17,9.0,"Counter({0: 90, 1: 81, 2: 77})"
5,0.13,0.18,"[46, 203, 311, 378, 379, 386, 430, 1392, 1906,...",0.30,"[3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, ...",0.31,0.18,55.0,"Counter({0: 82, 1: 78, 2: 49, 3: 39})"
6,0.15,0.64,"[7, 9, 26, 31, 34, 50, 51, 158, 261, 453, 818,...",0.66,"[1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, ...",0.31,0.30,37.0,"Counter({0: 87, 1: 84, 2: 77})"
7,0.12,0.23,"[2914, 4432, 5242, 7266, 8093, 10562, 12088]",0.26,"[2, 0, 1, 2, 2, 2, 1, 2, 1, 0, 2, 3, 1, 2, 1, ...",0.13,0.24,7.0,"Counter({0: 80, 2: 70, 1: 70, 3: 28})"
8,0.12,0.00,"[113, 132, 358, 432, 588, 597, 598, 762, 904, ...",0.02,"[0, 2, 1, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 0, 0, ...",0.23,0.12,63.0,"Counter({0: 91, 1: 87, 2: 70})"
9,0.16,0.10,"[15, 511, 534, 645, 770, 786, 817, 827, 845, 8...",0.11,"[0, 1, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, ...",0.49,0.19,152.0,"Counter({0: 94, 1: 90, 2: 64})"


adding 3 random population


100%|██████████| 31/31 [33:38<00:00, 65.11s/it] 

TIME: GA: 33.64013553063075 min
TIME: Total: 34.36368084351222 min


In [17]:
results.groupby("dataset_name").agg({"ari": max})

,ari
dataset_name,
alon,0.26
borovecki,0.75
chiaretti,0.28
christensen,0.88
golub,0.29
gordon,0.22
khan,0.81
sorlie,0.24
su,0.69


# Other methods

In [6]:
from sklearn import mixture
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
import hdbscan

## Silhouette analysis

In [23]:
results = pd.DataFrame()
filenames = np.array([
    'alon', 'borovecki', 'chiaretti', 'christensen', 'golub', 'gordon',
       'khan', 'sorlie', 'su', 'yeoh', 'west'
])
clustering = "hdbscan"
path = '../data/microarray/'

for name in filenames:
    print(name)
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values
    if data.shape[1] > 8000:
        data = data[:, :8000]
    n_clusters = len(np.unique(truth))
    row = {"dataset": name}
    clustering = AffinityPropagation(random_state=5).fit(data)
    ari = silhouette_score(data, clustering.labels_)
    print(f"Affinity {ari}")
    row["AffinityPropagation"] = ari

    clustering = SpectralClustering(n_clusters=n_clusters, assign_labels='discretize',random_state=5).fit(data)
    ari = silhouette_score(data, clustering.labels_)
    print(f"Spectral {ari}")
    row["Spectral"] = ari

    clustering = KMeans(n_clusters=n_clusters,random_state=5).fit(data)
    ari = silhouette_score(data, clustering.labels_)
    print(f"KMeans {ari}")
    row["KMeans"] = ari

    gmm = mixture.GaussianMixture(n_components=n_clusters,
                  covariance_type="full", random_state=0)
    pred = gmm.fit_predict(data)
    ari = silhouette_score(data, pred)
    print(f"GMM {ari}")
    row["GMM"] = ari

    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(data).labels_
    ari = silhouette_score(data, pred)
    print(f"HDBSCAN {ari}")
    row["HDBSCAN"] = ari

    pca = PCA(2)
    pca_data = pca.fit_transform(data)

    clustering = KMeans(n_clusters=n_clusters,random_state=5).fit(pca_data)
    ari = silhouette_score(pca_data, clustering.labels_)
    print(f"PCA KMeans {ari}")
    row["PCA_KMeans"] = ari

    gmm = mixture.GaussianMixture(n_components=n_clusters,
                  covariance_type="full", random_state=0)
    pred = gmm.fit_predict(pca_data)
    ari = silhouette_score(pca_data, pred)
    print(f"PCA GMM {ari}")
    row["PCA_GMM"] = ari

    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(pca_data).labels_
    ari = silhouette_score(pca_data, pred)
    print(f"PCAHDBSCAN {ari}")
    row["PCA_HDBSCAN"] = ari

    results = results.append(row, ignore_index = True)
    results.to_pickle("../data/microarray_others_silhouette.pkl")
# results.groupby("dataset_name").agg({"ari": max})

alon
Affinity 0.062115676466571706
Spectral 0.18971104380145723
KMeans 0.33751924055238436
GMM 0.39038081738376745
HDBSCAN 0.327951038200559
PCA KMeans 0.5386360867267125
PCA GMM 0.5948486954752348
PCAHDBSCAN 0.5404045400966923
borovecki
Affinity 0.05732736809638543
Spectral 0.00020361049546127082


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


KMeans 0.1834516193513604
GMM 0.16428416398565662
HDBSCAN -0.06528039250291505
PCA KMeans 0.5278758557385943
PCA GMM 0.4183195014864457
PCAHDBSCAN 0.2938380365229793
chiaretti
Affinity 0.05829586917319028
Spectral -0.06849106206603138
KMeans 0.08400771636709349
GMM 0.05976823400715967
HDBSCAN -0.0761508051875368
PCA KMeans 0.3891652629573138
PCA GMM 0.3702614925457201
PCAHDBSCAN 0.09342965822113936
christensen
Affinity 0.14378879241920922
Spectral 0.2505581978871958
KMeans 0.28698602009245067
GMM 0.28698602009245067
HDBSCAN 0.24138629251455723
PCA KMeans 0.635316384955393
PCA GMM 0.6323162377395329
PCAHDBSCAN 0.4499280716469588
golub
Affinity 0.046579455330318376


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Spectral 0.0246193242871702
KMeans 0.10296167977036548
GMM 0.10895234009293128
HDBSCAN 0.08005591489837682
PCA KMeans 0.41240509079013205
PCA GMM 0.3530838176490845
PCAHDBSCAN 0.2739315849058706
gordon
Affinity 0.016147481335841395


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Spectral 0.001220701781178454
KMeans 0.3276695862574315
GMM 0.08769326199312157
HDBSCAN -0.06911562830740205
PCA KMeans 0.6610834192367447
PCA GMM 0.5437945033279933
PCAHDBSCAN 0.15441864020098203
khan
Affinity 0.11536604899800153
Spectral 0.0373203269924958
KMeans 0.11498315455746094
GMM 0.11283131411052932
HDBSCAN 0.0872752723621908
PCA KMeans 0.5941763302797652
PCA GMM 0.5950128121511599
PCAHDBSCAN 0.42798402218891385
sorlie
Affinity 0.05754429031630421
Spectral 0.06661410352474696
KMeans 0.07195986313084812
GMM 0.054763539099721176
HDBSCAN 0.014795226629159946
PCA KMeans 0.4313279230991947
PCA GMM 0.358767713621992
PCAHDBSCAN 0.236830287626471
su
Affinity 0.11763943313320828
Spectral -0.00257658517967443
KMeans 0.14317786635918256
GMM 0.13088794750380037
HDBSCAN 0.07990960846641203
PCA KMeans 0.4989525314345633
PCA GMM 0.4423240646495514
PCAHDBSCAN 0.33114332565224175
yeoh
Affinity 0.033839258201076174
Spectral -0.07516042328023195
KMeans 0.03612338401004898
GMM 0.02984745557222693

# ARI analysis

In [8]:
results = pd.DataFrame()
filenames = np.array([
    'west','alon', 'borovecki', 'chiaretti', 'christensen', 'golub', 'gordon',
       'khan', 'sorlie', 'su', 'yeoh', ])
clustering = "hdbscan"
path = '../data/microarray/'
method = "adapted_ratkowsky_lance"

for name in filenames:
    print(name)
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values
    if data.shape[1] > 8000:
        data = data[:, :8000]
    n_clusters = len(np.unique(truth))
    row = {"dataset": name}
    clustering = AffinityPropagation(random_state=5).fit(data)
    ari = adjusted_rand_score(truth, clustering.labels_)
    print(f"Affinity {ari}")
    row["AffinityPropagation"] = ari

    clustering = SpectralClustering(n_clusters=n_clusters, assign_labels='discretize',random_state=5).fit(data)
    ari = adjusted_rand_score(truth, clustering.labels_)
    print(f"Spectral {ari}")
    row["Spectral"] = ari

    clustering = KMeans(n_clusters=n_clusters,random_state=5).fit(data)
    ari = adjusted_rand_score(truth, clustering.labels_)
    print(f"KMeans {ari}")
    row["KMeans"] = ari

    gmm = mixture.GaussianMixture(n_components=n_clusters,
                  covariance_type="full", random_state=0)
    pred = gmm.fit_predict(data)
    ari = adjusted_rand_score(truth, pred)
    print(f"GMM {ari}")
    row["GMM"] = ari

    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(data).labels_
    ari = adjusted_rand_score(truth, pred)
    print(f"HDBSCAN {ari}")
    row["HDBSCAN"] = ari

    pca = PCA(2)
    pca_data = pca.fit_transform(data)

    clustering = KMeans(n_clusters=n_clusters,random_state=5).fit(pca_data)
    ari = adjusted_rand_score(truth, clustering.labels_)
    print(f"PCA KMeans {ari}")
    row["PCA_KMeans"] = ari

    gmm = mixture.GaussianMixture(n_components=n_clusters,
                  covariance_type="full", random_state=0)
    pred = gmm.fit_predict(pca_data)
    ari = adjusted_rand_score(truth, pred)
    print(f"PCA GMM {ari}")
    row["PCA_GMM"] = ari

    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(pca_data).labels_
    ari = adjusted_rand_score(truth, pred)
    print(f"PCAHDBSCAN {ari}")
    row["PCA_HDBSCAN"] = ari

    results = results.append(row, ignore_index = True)
    results.to_pickle("../data/microarray_others.pkl")
# results.groupby("dataset_name").agg({"ari": max})

west
Affinity 0.010971235670581616
Spectral -0.015456057665650124
KMeans -0.0041809458533241944


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


GMM 0.014631440260730879
HDBSCAN -0.0013206366858969903
PCA KMeans 0.0039014373716632442
PCA GMM -0.008853201564751905
PCAHDBSCAN 0.028903153390157494
alon
Affinity 0.06114066313750697
Spectral -0.00965753681291918
KMeans -0.01832753338552532
GMM -0.041705878302384834
HDBSCAN -0.07204023223578718
PCA KMeans -0.01832753338552532
PCA GMM -0.04814177497890322
PCAHDBSCAN -0.06776579509080834
borovecki
Affinity 0.3498443354896441
Spectral -0.03178565485105675
KMeans 0.15717685829366532


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


GMM 0.21518987341772153
HDBSCAN 0.15651723244363824
PCA KMeans 0.10807255987285581
PCA GMM 0.27686753679533466
PCAHDBSCAN 0.18848167539267016
chiaretti
Affinity 0.002067838978669255
Spectral -0.014478552554577664
KMeans -0.01183474044815515
GMM 0.011690399378572447
HDBSCAN -0.021070423525464545
PCA KMeans -0.013216039768117636
PCA GMM -0.019860864107804122
PCAHDBSCAN -0.01985584386721365
christensen
Affinity 0.28079936738945627
Spectral 0.20488269028642028
KMeans 0.9828429301705416
GMM 0.9828429301705416
HDBSCAN 0.44599263739995926
PCA KMeans 0.9492317112192555
PCA GMM 0.966345822674811
PCAHDBSCAN 0.48591682416507553
golub
Affinity 0.09794745851353995


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Spectral 0.02437970235508023
KMeans 0.16207751501386602
GMM 0.2111546312131075
HDBSCAN 0.052535013845996056
PCA KMeans 0.2111546312131075
PCA GMM -0.05815801223119611
PCAHDBSCAN 0.03254154258020062
gordon
Affinity 0.06443032737658316


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Spectral 0.0034126343792930504
KMeans 0.4094357339241926
GMM 0.019886908184780526
HDBSCAN -0.05999462161864832
PCA KMeans 0.3913274977531081
PCA GMM 0.8122761573399956
PCAHDBSCAN 0.03551395193364228
khan
Affinity 0.2192505201591399
Spectral 0.23527599677099087
KMeans 0.06595712786688653
GMM 0.04565391299550357
HDBSCAN 0.19650287185934123
PCA KMeans 0.05299278573331886
PCA GMM 0.06584227303680164
PCAHDBSCAN 0.10001565054081313
sorlie
Affinity 0.3588011382534462
Spectral 0.20883997372679067
KMeans 0.4472746039398899
GMM 0.3752708372881578
HDBSCAN -0.0013327267128984968
PCA KMeans 0.43496208761528765
PCA GMM 0.5961412258826617
PCAHDBSCAN 0.2090726581521507
su
Affinity 0.5717241331174441
Spectral 0.08790084186243431
KMeans 0.8965943968314738
GMM 0.6391711442977579
HDBSCAN 0.7117296249855959
PCA KMeans 0.4162409917380749
PCA GMM 0.3320029787900224
PCAHDBSCAN 0.21515948349438987
yeoh
Affinity 0.18333811544096054
Spectral 0.03656635514018691
KMeans 0.0990670103519137
GMM 0.02909912125417917
H

In [5]:
results.round(2)

,AffinityPropagation,GMM,HDBSCAN,KMeans,PCA_GMM,PCA_HDBSCAN,PCA_KMeans,Spectral,dataset
0,0.06,-0.04,-0.07,-0.02,-0.05,-0.07,-0.02,-0.01,alon
1,0.35,0.22,0.16,0.16,0.28,0.19,0.11,-0.03,borovecki
2,0.00,0.01,-0.02,-0.01,-0.02,-0.02,-0.01,-0.01,chiaretti
3,0.28,0.98,0.45,0.98,0.97,0.49,0.95,0.20,christensen
4,0.10,0.21,0.05,0.16,-0.06,0.03,0.21,0.02,golub
5,0.06,0.02,-0.06,0.41,0.81,0.03,0.39,0.00,gordon
6,0.22,0.05,0.20,0.07,0.07,0.10,0.05,0.24,khan
7,0.36,0.38,-0.00,0.45,0.60,0.21,0.43,0.21,sorlie
8,0.57,0.64,0.71,0.90,0.33,0.22,0.42,0.09,su
9,0.18,0.03,-0.00,0.10,0.02,-0.00,0.02,0.04,yeoh


# Supervised analysis of datasets

In [7]:
import scripts.ga_evaluation as ga_evaluation
filenames = np.array(['alon', 'borovecki', 'chiaretti', 'christensen', 'golub', 'gordon',
       'khan', 'sorlie', 'su', 'yeoh'])
path = '../data/microarray/'
imp_f = np.arange(20)
result_df = pd.DataFrame()
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values

    n_clusters = len(np.unique(truth))

    result = {"Dataset" : name,
             "Original Dimensions": f"{data.shape[0]} x {data.shape[1]}",
             "Cluster sizes" : ", ".join(np.array(list(Counter(truth).values())).astype(str))}
    
    predK = KMeans(n_clusters= n_clusters, random_state = 2).fit(data).labels_
    ari_all = adjusted_rand_score(truth, predK)
    
    predK = KMeans(n_clusters= n_clusters, random_state = 2).fit(data[:, :5]).labels_
    ari_top10 = adjusted_rand_score(truth, predK)
    
    pca = PCA(2)
    pca_data = pca.fit_transform(data)

    predK = KMeans(n_clusters= n_clusters, random_state = 2).fit(pca_data).labels_
    ari_pca = adjusted_rand_score(truth, predK)
    
    r1 = ga_evaluation.random_sampling(data, truth, n_clusters, algo = "gmm")
    r2 = ga_evaluation.random_sampling(data, truth, n_clusters, algo = "hdbscan")
    result["ARI all dataset"] = round(ari_all,2)
    result["ARI PCA dataset"] = round(ari_pca,2)
    result["ARI top 10 features"] = round(ari_top10,2)
    result["Random GMM"] = round(r1,2)
    result["Random HDBSCAN"] = round(r2,2)
    result_df = result_df.append(result, ignore_index = True)

result_df

100%|██████████| 100/100 [00:01<00:00, 85.06it/s]


,ARI PCA dataset,ARI all dataset,ARI top 10 features,Cluster sizes,Dataset,Original Dimensions,Random GMM,Random HDBSCAN
0,-0.02,-0.02,0.01,"40, 22",alon,62 x 1991,-0.03,0.00
1,0.35,0.54,0.75,"17, 14",borovecki,31 x 22283,0.10,0.00
2,-0.01,0.03,0.28,"37, 74, 10, 5, 1, 1",chiaretti,128 x 12625,0.01,0.00
3,0.95,0.98,0.61,"113, 85, 19",christensen,217 x 1413,0.55,0.16
4,0.21,0.24,0.79,"47, 25",golub,72 x 7129,0.03,0.00
5,0.39,0.61,0.92,"31, 150",gordon,181 x 12533,0.22,0.00
6,0.07,0.08,1.00,"23, 20, 12, 8",khan,63 x 2308,0.10,0.00
7,0.43,0.55,0.40,"14, 11, 13, 15, 32",sorlie,85 x 456,0.13,0.00
8,0.42,0.90,0.83,"25, 26, 28, 23",su,102 x 5563,0.24,0.00
9,0.01,0.09,0.63,"15, 27, 64, 20, 43, 79",yeoh,248 x 12625,0.04,0.00


# Run time analysis

In [ ]:
results = pd.read_excel("reports/r_cnn.xlsx", index_col=0)

In [ ]:
results["label"] = results["dataset_name"] + " (" +results["input_size"].astype(str) + " features) " 

In [ ]:
perf = results.groupby("input_size").min()[['t(feature_sel)', 't(cnn)','t(ga)']]

In [ ]:
perf = perf.rename(columns = {
    't(feature_sel)' : '1D Feature ranking', 
    't(cnn)' : '2D Feature ranking with NN',
    't(ga)': 'Optimization algorithm for top 10 subspaces'
})

In [ ]:
plt.figure(figsize = (10,3))
ax = plt.gca()
perf.plot(kind='bar', stacked=True, ax = ax)
plt.ylabel("time (min)")
plt.xticks(rotation = 0)
sns.despine()
plt.title("Run times on West, Khan, Gordon and Boroveki datasets")
plt.xlabel("Number of dimensions in the input dataset")
plt.savefig(f"images/run_times.pdf", bbox_inches='tight')

# Best scores using supervised feature selection

In [8]:
import scripts.ga_evaluation as ga_evaluation
from sklearn import mixture
import hdbscan
filenames = np.array(['alon', 'borovecki', 'chiaretti', 'christensen', 'golub', 'gordon',
       'khan', 'sorlie', 'su', 'yeoh', 'west'])
path = '../data/microarray/'

In [9]:
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values
    print(Counter(truth))
    n_clusters = len(np.unique(truth))
    gmm_scores = []
    hdbscan_scores = []
    for i in range(2, 50):
        input_data = data[:, :i]
        gmm = mixture.GaussianMixture(n_components=n_clusters,
                          covariance_type="full", random_state=0)
        pred = gmm.fit_predict(input_data)
        ari = adjusted_rand_score(truth, pred)
        gmm_scores.append(ari)

        pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
        ari = adjusted_rand_score(truth, pred)
        hdbscan_scores.append(ari)

        
    print(f"\n\n\n{name} GMM ari = {max(gmm_scores)}, ")
    print(f"{name} HDBSCAN ari = {max(hdbscan_scores)}, ")
    

Counter({1: 40, 0: 22})



alon GMM ari = 0.5408956029134071, 
alon HDBSCAN ari = 0.2614104988950006, 
Counter({1: 17, 0: 14})



borovecki GMM ari = 0.7506518000776613, 
borovecki HDBSCAN ari = 0.442523768366465, 
Counter({3: 74, 1: 37, 0: 10, 2: 5, 5: 1, 4: 1})



chiaretti GMM ari = 0.5027786102396937, 
chiaretti HDBSCAN ari = 0.38195418120210867, 
Counter({0: 113, 1: 85, 2: 19})



christensen GMM ari = 1.0, 
christensen HDBSCAN ari = 0.9662417166002536, 
Counter({0: 47, 1: 25})



golub GMM ari = 1.0, 
golub HDBSCAN ari = 0.6696342951791474, 
Counter({0: 150, 1: 31})



gordon GMM ari = 1.0, 
gordon HDBSCAN ari = 0.8919758343184441, 
Counter({1: 23, 3: 20, 2: 12, 0: 8})



khan GMM ari = 1.0, 
khan HDBSCAN ari = 0.9857833454105082, 
Counter({4: 32, 3: 15, 0: 14, 2: 13, 1: 11})



sorlie GMM ari = 0.7633359805059493, 
sorlie HDBSCAN ari = 0.29016918700404254, 
Counter({2: 28, 1: 26, 0: 25, 3: 23})



su GMM ari = 0.9752497571716532, 
su HDBSCAN ari = 0.8788950801065628, 
Counter({5

# Mutual Information

In [10]:
from sklearn.feature_selection import chi2,  mutual_info_classif, SelectKBest

In [11]:
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values
    print(Counter(truth))
    n_clusters = len(np.unique(truth))
    gmm_scores = []
    hdbscan_scores = []
    sel = SelectKBest(mutual_info_classif, k=50).fit_transform(data, truth)
    for i in range(2, 50):
        input_data = sel[:, :i]
        gmm = mixture.GaussianMixture(n_components=n_clusters,
                          covariance_type="full", random_state=0)
        pred = gmm.fit_predict(input_data)
        ari = adjusted_rand_score(truth, pred)
        gmm_scores.append(ari)

        pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
        ari = adjusted_rand_score(truth, pred)
        hdbscan_scores.append(ari)

        
    print(f"\n\n\n{name} GMM ari = {max(gmm_scores)}, ")
    print(f"{name} HDBSCAN ari = {max(hdbscan_scores)}, ")

Counter({1: 40, 0: 22})



alon GMM ari = 0.4473743368402122, 
alon HDBSCAN ari = 0.2614104988950006, 
Counter({1: 17, 0: 14})



borovecki GMM ari = 1.0, 
borovecki HDBSCAN ari = 0.775027912169706, 
Counter({3: 74, 1: 37, 0: 10, 2: 5, 5: 1, 4: 1})



chiaretti GMM ari = 0.3627043165289317, 
chiaretti HDBSCAN ari = 0.24742717368780276, 
Counter({0: 113, 1: 85, 2: 19})



christensen GMM ari = 0.9773124968542636, 
christensen HDBSCAN ari = 0.817866868130873, 
Counter({0: 47, 1: 25})



golub GMM ari = 0.9439994667673883, 
golub HDBSCAN ari = 0.6696342951791474, 
Counter({0: 150, 1: 31})



gordon GMM ari = 1.0, 
gordon HDBSCAN ari = 0.9060143786845584, 
Counter({1: 23, 3: 20, 2: 12, 0: 8})



khan GMM ari = 1.0, 
khan HDBSCAN ari = 0.9753291540426958, 
Counter({4: 32, 3: 15, 0: 14, 2: 13, 1: 11})



sorlie GMM ari = 0.7753005995131064, 
sorlie HDBSCAN ari = 0.22462124648462306, 
Counter({2: 28, 1: 26, 0: 25, 3: 23})



su GMM ari = 1.0, 
su HDBSCAN ari = 0.3695008593957911, 
Counter({5:

In [ ]:
import scripts.ga_evaluation as ga_evaluation
filenames = np.array(['alon', 'borovecki', 'chiaretti', 'christensen', 'golub', 'gordon',
       'khan', 'sorlie', 'su', 'yeoh', 'west'])
path = '../data/microarray/'

for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    input_data = data.drop("truth", axis = 1).values
    if len(input_data) > 5000:
        input_data = input_data[:, :5000]

    n_clusters = len(np.unique(truth))
    gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
    pred = gmm.fit_predict(input_data)
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} GMM ari = {ari}")

    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} HDBSCAN ari = {ari}")

alon GMM ari = -0.041705878302384834
alon HDBSCAN ari = -0.07204023223578718


In [ ]:
import scripts.ga_evaluation as ga_evaluation
filenames = np.array(['borovecki'])
path = '../data/microarray/'

for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    input_data = data.drop("truth", axis = 1).values
    input_data = input_data[:, :10000]
    

    n_clusters = len(np.unique(truth))
    
    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} HDBSCAN ari = {ari}")
    
    gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
    pred = gmm.fit_predict(input_data)
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} GMM ari = {ari}")




# Execution time analysis


In [ ]:
results = pd.read_excel("reports/r_cnn.xlsx", index_col=0)

results["label"] = results["dataset_name"] + " (" +results["input_size"].astype(str) + " features) " 

perf = results.groupby("input_size").min()[['t(feature_sel)', 't(cnn)','t(ga)']]

perf = perf.rename(columns = {
    't(feature_sel)' : '1D Feature ranking', 
    't(cnn)' : '2D Feature ranking with NN',
    't(ga)': 'Optimization algorithm for top 10 subspaces'
})



plt.figure(figsize = (10,3))
ax = plt.gca()
perf.plot(kind='bar', stacked=True, ax = ax)
plt.ylabel("time (min)")
plt.xticks(rotation = 0)
sns.despine()
plt.title("Run times on West, Khan, Gordon and Boroveki datasets")
plt.xlabel("Number of dimensions in the input dataset")
plt.savefig(f"images/run_times.pdf", bbox_inches='tight')